In [ ]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

cmd = cmdgen.CommandGenerator()

In [ ]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.3.6.1.2.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

chexiang
===

- system = 1.3.6.1.2.1.1
- sysName = 1.3.6.1.2.1.1.5
- ifEntry = 1.3.6.1.2.1.2.2.1
- ifDescr = 1.3.6.1.2.1.2.2.1.2
- ifSpeed = 1.3.6.1.2.1.2.2.1.5

## 同步

In [ ]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
cmdGen = cmdgen.CommandGenerator()  # 同步

mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
)
mibView = view.MibViewController(mibBuilder)



errorIndication, errorStatus, errorIndex, varBinds = cmdGen.nextCmd(
    cmdgen.CommunityData('huawei@123'),
    cmdgen.UdpTransportTarget(('10.32.33.16', 161)),
    cmdgen.MibVariable('IF-MIB', 'ifOperStatus')
)


print(errorIndication)
print(errorStatus)
print(errorIndex)
print('*' * 50)

for row in varBinds:
    for oid, val in row:
        print(oid.getMibSymbol()[-1][0], val.prettyPrint())

## 异步

In [ ]:
import string
import time
# from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
# cmdGen = cmdgen.CommandGenerator()  # 同步
cmdGen = cmdgen.AsynCommandGenerator()  # 异步

# mibBuilder = builder.MibBuilder()
# mibBuilder.loadModules(
#     'RFC-1213', 'RFC-3413'
# )
# mibView = view.MibViewController(mibBuilder)


mib_vars = (
#     {'desc': 'ifOperStatus', 'mib': cmdgen.MibVariable('IF-MIB', 'ifOperStatus')},  # 端口状态
#     {'desc': 'ifSpeed', 'mib': cmdgen.MibVariable('IF-MIB', 'ifSpeed')},  # 端口速度 bits/s
#     {'desc': 'ifDescr', 'mib': cmdgen.MibVariable('IF-MIB', 'ifDescr')},
#     {'desc': 'ifInOctets', 'mib': cmdgen.MibVariable('IF-MIB', 'ifInOctets')},
#     {'desc': 'ifOutOctets', 'mib': cmdgen.MibVariable('IF-MIB', 'ifOutOctets')},
    {'desc': 'multi', 'mibs': (cmdgen.MibVariable('IF-MIB', 'ifDescr'), cmdgen.MibVariable('IF-MIB', 'ifSpeed'))},

)


def callback(sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
    if errorIndication:
        print(errorIndication)
        print(errorStatus)
        print(errorIndex)
        return 1
        
    for row in varBinds:
        for oid, val in row:
            c = 0
            if cbCtx['object_name'] not in oid.prettyPrint():
#                 print(oid.prettyPrint())
                c+=1
                if c > 200:
                    return
#                 return
            
            print(oid.getLabel()[-1], str(int(oid.getMibSymbol()[-1][0])), val.prettyPrint())
            
    return 1  # 告诉 walk 继续运行
            


for mib_var in mib_vars:
    cbCtx = {'object_name': mib_var['desc']}
    cmdGen.nextCmd(
        cmdgen.CommunityData('huawei@123'),
        cmdgen.UdpTransportTarget(('10.32.33.16', 161), timeout=1, retries=3),
#         0, 25,
#         (mib_var['mib'],),
        mib_var['mibs'],
        (callback, cbCtx),
        True, True
    )

    
# 启动
print('start job')
# cmdGen.snmpEngine.transportDispatcher.jobStarted(1)
print('run dispatcher')
cmdGen.snmpEngine.transportDispatcher.runDispatcher()
print('close dispatcher')
# cmdGen.snmpEngine.transportDispatcher.closeDispatcher()
print('job finished')
# cmdGen.snmpEngine.transportDispatcher.
# cmdGen.snmpEngine.transportDispatcher.jobFinished(1)
